In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "pkl/uber-raw-data-combined.csv.pkl"
df = pd.read_pickle(file)
df

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [3]:
def time_sampling(start, end):
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
    filter_df = df[(df["Date/Time"] >= start) & (df["Date/Time"] <= end)]
    sample_df = filter_df.sort_values(by="Date/Time")
    return sample_df

### Inputs go by:
"Year-Month-Day Hour:Minute:Second"

*time is in 24 hour clock

the rows are out of order because original csv was sorted by base values, not time.

In [4]:
#time_sampling("2014-04-01 00:00:00", "2014-04-29 00:00:00")

In [12]:
if (df["Date/Time"].dtypes != np.datetime64):
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
print(f"Max: {df.max()}")
print(f"Min: {df.min()}")
lat = df["Lat"].sort_values().unique()
lon = df["Lon"].sort_values().unique()
time = df["Date/Time"].sort_values().unique()

# tensor = np.zeros([len(lat), len(lon), len(time)])
lat, lon, time

Max: Date/Time    2014-09-30 22:59:00
Lat                      42.1166
Lon                     -72.0666
Base                      B02764
dtype: object
Min: Date/Time    2014-04-01 00:00:00
Lat                      39.6569
Lon                      -74.929
Base                      B02512
dtype: object


(array([39.6569, 39.6686, 39.7214, ..., 41.5016, 41.5276, 42.1166]),
 array([-74.929 , -74.8594, -74.8575, ..., -72.2999, -72.1801, -72.0666]),
 <DatetimeArray>
 ['2014-04-01 00:00:00', '2014-04-01 00:01:00', '2014-04-01 00:02:00',
  '2014-04-01 00:03:00', '2014-04-01 00:04:00', '2014-04-01 00:05:00',
  '2014-04-01 00:06:00', '2014-04-01 00:07:00', '2014-04-01 00:08:00',
  '2014-04-01 00:09:00',
  ...
  '2014-09-30 22:50:00', '2014-09-30 22:51:00', '2014-09-30 22:52:00',
  '2014-09-30 22:53:00', '2014-09-30 22:54:00', '2014-09-30 22:55:00',
  '2014-09-30 22:56:00', '2014-09-30 22:57:00', '2014-09-30 22:58:00',
  '2014-09-30 22:59:00']
 Length: 260093, dtype: datetime64[ns])